In [1]:
import pandas as pd

# 각 파일을 로드
average_rsi_df = pd.read_csv('C:/Users/82106/Desktop/경영 캡스톤 프로젝트2/average_rsi_df.csv')
average_volumes = pd.read_csv('C:/Users/82106/Desktop/경영 캡스톤 프로젝트2/average_volumes.csv')
df_beta_values = pd.read_csv('C:/Users/82106/Desktop/경영 캡스톤 프로젝트2/df_beta_values.csv')
dividend_yields = pd.read_csv('C:/Users/82106/Desktop/경영 캡스톤 프로젝트2/dividend_yields.csv')
per_values = pd.read_csv('C:/Users/82106/Desktop/경영 캡스톤 프로젝트2/per_values.csv')
volatility_df = pd.read_csv('C:/Users/82106/Desktop/경영 캡스톤 프로젝트2/volatility_df.csv')

# 모든 데이터를 'Ticker' 기준으로 병합
data_frames = [average_rsi_df, average_volumes, df_beta_values, dividend_yields, per_values, volatility_df]
merged_df = pd.concat(data_frames, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]  # 중복 열 제거

# 병합된 데이터 프레임 출력
merged_df.head()


,Ticker,RSI,0,Beta,DividendYield,PER,Volatility
0,AAPL,60.891036,6.888900e+07,1.264,0.55,{'PER': 28.970451},0.012570
1,MSFT,61.321318,6.343245e+06,0.893,0.72,{'PER': 35.88205},0.015824
2,AMZN,61.228543,5.172203e+06,1.155,NaN,{'PER': 52.407307},0.020805
3,FB,NaN,2.416384e+06,NaN,NaN,{'PER': None},NaN
4,GOOGL,53.160397,3.051076e+06,1.019,0.47,{'PER': 25.901989},0.019149


### 배당수익률이 NaN인 행은 0으로 대체

In [2]:
# 'DividendYield' 열의 결측치를 0으로 대체
merged_df['DividendYield'] = merged_df['DividendYield'].fillna(0)

In [3]:
merged_df.rename(columns={'0': 'volume'}, inplace=True)

### 6개열에서 결측치가 한개라도 있는 행은 삭제

In [4]:
# 결측치가 있는 행 삭제
cleaned_df = merged_df.dropna(subset=['PER', 'DividendYield', 'Beta','RSI','volume','Volatility'])

# 결과 확인
print(cleaned_df)

   Ticker        RSI        volume   Beta  DividendYield                 PER  \
0    AAPL  60.891036  6.888900e+07  1.264           0.55  {'PER': 28.970451}   
1    MSFT  61.321318  6.343245e+06  0.893           0.72   {'PER': 35.88205}   
2    AMZN  61.228543  5.172203e+06  1.155           0.00  {'PER': 52.407307}   
4   GOOGL  53.160397  3.051076e+06  1.019           0.47  {'PER': 25.901989}   
5    GOOG  53.086364  3.502121e+06  1.019           0.47  {'PER': 26.211655}   
..    ...        ...           ...    ...            ...                 ...   
93    CME  51.810189  6.287800e+06  0.488           4.15  {'PER': 23.908884}   
94     CL  64.148240  2.044373e+07  0.394           2.10    {'PER': 29.9873}   
95    PLD  63.595179  1.963317e+07  1.058           3.57   {'PER': 31.48538}   
96   LRCX  62.697293  1.699392e+07  1.511           0.87   {'PER': 33.23979}   
97     SO  59.866619  2.172282e+06  0.476           3.69  {'PER': 20.388601}   

    Volatility  
0     0.012570  
1    

13개 행이 삭제됨

### 범주형 변수 인코딩

In [5]:
# 데이터 유형 확인
print("Data types before conversion:")
print(cleaned_df[['PER', 'DividendYield', 'Beta','RSI','volume','Volatility']].dtypes)

Data types before conversion:
PER               object
DividendYield    float64
Beta             float64
RSI              float64
volume           float64
Volatility       float64
dtype: object


PER의 { } 안의 수만 추출 하도록 가공

In [6]:
import ast
import pandas as pd

def extract_per_value(per_string):
    if pd.isna(per_string):
        return float('nan')  # 입력 값이 NaN인 경우, NaN 반환

    try:
        # 문자열을 사전으로 변환
        per_dict = ast.literal_eval(per_string)
        # 'PER' 키의 값을 float으로 변환하여 반환
        return float(per_dict['PER']) if per_dict['PER'] is not None else float('nan')
    except (ValueError, SyntaxError, KeyError, TypeError):
        # 변환에 실패한 경우 NaN 반환
        return float('nan')

# 'PER' 열을 숫자형으로 변환
cleaned_df['PER'] = cleaned_df['PER'].apply(extract_per_value)

# 결과 확인
print(cleaned_df[['Ticker', 'PER']])

   Ticker        PER
0    AAPL  28.970451
1    MSFT  35.882050
2    AMZN  52.407307
4   GOOGL  25.901989
5    GOOG  26.211655
..    ...        ...
93    CME  23.908884
94     CL  29.987300
95    PLD  31.485380
96   LRCX  33.239790
97     SO  20.388601

[96 rows x 2 columns]


C:\Users\82106\AppData\Local\Temp\ipykernel_40848\2944089155.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['PER'] = cleaned_df['PER'].apply(extract_per_value)


In [7]:
cleaned_df

,Ticker,RSI,volume,Beta,DividendYield,PER,Volatility
0,AAPL,60.891036,6.888900e+07,1.264,0.55,28.970451,0.012570
1,MSFT,61.321318,6.343245e+06,0.893,0.72,35.882050,0.015824
2,AMZN,61.228543,5.172203e+06,1.155,0.00,52.407307,0.020805
4,GOOGL,53.160397,3.051076e+06,1.019,0.47,25.901989,0.019149
5,GOOG,53.086364,3.502121e+06,1.019,0.47,26.211655,0.019283
...,...,...,...,...,...,...,...
93,CME,51.810189,6.287800e+06,0.488,4.15,23.908884,0.011325
94,CL,64.148240,2.044373e+07,0.394,2.10,29.987300,0.010355
95,PLD,63.595179,1.963317e+07,1.058,3.57,31.485380,0.016672
96,LRCX,62.697293,1.699392e+07,1.511,0.87,33.239790,0.022521


### 결측치 처리하기

In [8]:
# 'PER' 열에서 NaN 값을 가진 행을 식별
nan_per_tickers = cleaned_df[cleaned_df['PER'].isna()]['Ticker']

# 결과 출력
print("NaN 이 있는 티커:")
print(nan_per_tickers)


NaN 이 있는 티커:
20    PFE
40     BA
41    MMM
47    BMY
85     MU
Name: Ticker, dtype: object


In [9]:
# 'PER' 열에서 NaN 값이 있는 행을 삭제
cleaned_df = cleaned_df.dropna(subset=['PER'])

In [10]:
# 'PER' 열에서 NaN 값을 가진 행을 식별
nan_per_tickers = cleaned_df[cleaned_df['PER'].isna()]['Ticker']

# 결과 출력
print("NaN 이 있는 티커:")
print(nan_per_tickers)

NaN 이 있는 티커:
Series([], Name: Ticker, dtype: object)


In [11]:
# 저장 경로 설정
save_path = 'C:/Users/82106/Desktop/경영 캡스톤 프로젝트2/cleaned_data.csv'

# 데이터 프레임을 CSV 파일로 저장
cleaned_df.to_csv(save_path, index=False)